In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import Statements
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import gc
import os

In [3]:
# Importing each csv files
aisles = pd.read_csv('/content/drive/MyDrive/instacart-market-basket-analysis/aisles.csv').sample(frac=0.7,random_state=0)

department = pd.read_csv('/content/drive/MyDrive/instacart-market-basket-analysis/departments.csv').sample(frac=0.7,random_state=0)


orders = pd.read_csv('/content/drive/MyDrive/instacart-market-basket-analysis/orders.csv').sample(frac=0.7,random_state=0)


products = pd.read_csv('/content/drive/MyDrive/instacart-market-basket-analysis/products.csv').sample(frac=0.7,random_state=0)


prior = pd.read_csv('/content/drive/MyDrive/instacart-market-basket-analysis/order_products__prior.csv').sample(frac=0.7,random_state=0)


train = pd.read_csv('/content/drive/MyDrive/instacart-market-basket-analysis/order_products__train.csv').sample(frac=0.7,random_state=0)


submission = pd.read_csv('/content/drive/MyDrive/instacart-market-basket-analysis/sample_submission.csv').sample(frac=0.7,random_state=0)


In [4]:
# Will see the shape of each dataframe and will decide whether to drop some rows or not
print("Aisles DataFrame",aisles.shape)
print("Department DataFrame",department.shape)
print("Orders DataFrame",orders.shape)
print("Products DataFrame",products.shape)
print("Order Products Prior DataFrame",prior.shape)
print("Order Products Train DataFrame",train.shape)
print("Submission DataFrame",submission.shape)

Aisles DataFrame (94, 2)
Department DataFrame (15, 2)
Orders DataFrame (2394758, 7)
Products DataFrame (34782, 4)
Order Products Prior DataFrame (22704142, 4)
Order Products Train DataFrame (969232, 4)
Submission DataFrame (52500, 2)


In [5]:
orders.head(2)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
2356430,3148837,141850,prior,23,1,14,5.0
2777371,2196491,167442,prior,2,4,18,4.0


In [6]:
prior.head(2)

,order_id,product_id,add_to_cart_order,reordered
8044718,849196,39332,1,0
30839208,3252968,45679,2,1


In [7]:
train.head(2)

,order_id,product_id,add_to_cart_order,reordered
276026,672376,49235,10,1
1266994,3130165,4146,2,1


In [8]:
# Since there are many NaN values in 'days_since_prior_order' so we do mean imputation for them
# Below code finds the mean of the 'days_since_prior_order

prior_order = list()
for key,value in enumerate(orders['days_since_prior_order'].values):
    if(str(value)=='nan'):
      flag=1
    else:  
      prior_order.append(value)
mean_value = sum(prior_order)/len(prior_order)
print("Mean Value Of Days Since Prior Order",np.round(mean_value,1))

# Replaces NaN with mean of 'days_since_prior_order'

prior_order = list()
null_count = 0
mean_values = np.round(mean_value,1)
for key,value in enumerate(orders['days_since_prior_order'].values):
    if(str(value)=='nan'):
        prior_order.append(mean_values)
        null_count+=1
    else:
        prior_order.append(value)           
print("Total Null Values were", null_count)        

orders.drop('days_since_prior_order', axis=1, inplace=True)
orders['days_since_prior_order'] = prior_order

Mean Value Of Days Since Prior Order 11.1
Total Null Values were 144685


In [9]:
prior_order_df = pd.merge(orders, prior, on='order_id', how='inner')
prior_order_df.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,3148837,141850,prior,23,1,14,5.0,47209,3,1
1,3148837,141850,prior,23,1,14,5.0,39619,2,1
2,3148837,141850,prior,23,1,14,5.0,21709,4,1


<h2> Creating User Features </h2>

1. Maximum number of order by user.
2. Mean number of order by user.
3. Minimum number of order by user.

In [10]:
user_df = pd.DataFrame()
user_df['max_number_of_orders'] = prior_order_df.groupby(by='user_id')['order_number'].aggregate('max').astype(np.uint8)
user_df['mean_number_of_orders'] = prior_order_df.groupby(by='user_id')['order_number'].aggregate('mean').astype(np.uint8)
user_df['min_number_of_orders'] = prior_order_df.groupby(by='user_id')['order_number'].aggregate('min').astype(np.uint8)
print("Null values", user_df.isnull().any().sum().sum())
user_df.head(3)

Null values 0


,max_number_of_orders,mean_number_of_orders,min_number_of_orders
user_id,,,
1,10,5,1
2,14,7,1
3,12,7,2


<h2> Creating Max Order Features </h2>

1. Average product by user in each order

In [11]:
max_order = pd.DataFrame()
average_products= pd.DataFrame()
max_order['total_products_per_order'] = prior_order_df.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').astype(np.uint8)
average_products['average_user_product'] = max_order.groupby(by=['user_id'])['total_products_per_order'].aggregate('mean').astype(np.float16)

print("Null values", average_products.isnull().any().sum().sum())
average_products.head(3)

Null values 0


,average_user_product
user_id,
1,4.222656
2,10.000000
3,5.144531


<h2> Day Of Week Features</h2>

1. Maximum order by user day wise

In [12]:
day_of_week = pd.DataFrame()
day_of_week['max_order_day'] = prior_order_df.groupby(by=['user_id'])['order_dow'].aggregate(lambda x : stats.mode(x)[0]).astype(np.uint8)
day_of_week = day_of_week.reset_index()
print("Null values", day_of_week.isnull().any().sum().sum())
day_of_week.head()

Null values 0


,user_id,max_order_day
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


<h2>Hour Of Day Feature</h2>

1. Hour with maximum order by user 

In [13]:
hour_of_day = pd.DataFrame()
hour_of_day['max_order_by_hour'] = prior_order_df.groupby(by=['user_id'])['order_hour_of_day'].aggregate(lambda x : stats.mode(x)[0]).astype(np.uint8)
hour_of_day = hour_of_day.reset_index()
print("Total NUll Values",hour_of_day.isnull().any().sum().sum())
hour_of_day.head(3)

Total NUll Values 0


,user_id,max_order_by_hour
0,1,8
1,2,9
2,3,16


<h2>Reorder Features</h2>

1. Total Number Of Reorders.
2. Total Number Of Reorders Not Done.
3. Reorder Ratio For Each User

In [14]:
reorder_by_user = pd.DataFrame()
reorder_by_user['total_reorders'] = prior_order_df[prior_order_df['reordered']==1].groupby('user_id')['reordered'].aggregate('count').astype(np.uint8)
reorder_by_user['total_non_reorders'] = prior_order_df[prior_order_df['reordered']==0].groupby('user_id')['reordered'].aggregate('count').astype(np.uint8)
reorder_by_user['reorder_ratio'] = reorder_by_user['total_reorders']/(reorder_by_user['total_reorders'] + reorder_by_user['total_non_reorders']).astype(np.float16)
print("Total NUll Values",reorder_by_user.isnull().any().sum().sum())
reorder_by_user.head(3)

Total NUll Values 2


,total_reorders,total_non_reorders,reorder_ratio
user_id,,,
1,27,11.0,0.710449
2,52,58.0,0.472656
3,27,9.0,0.750000


<h2>Merging All Features Created Till Now On 'User_id'</h2>

In [15]:
user_df = pd.merge(user_df, average_products, on='user_id', how='left')  
user_df = pd.merge(user_df, day_of_week, on='user_id', how='left')
user_df = pd.merge(user_df, hour_of_day, on='user_id', how='left')
user_df = pd.merge(user_df, reorder_by_user, on='user_id', how='left')
print("Total null values", user_df.isnull().any().sum())
user_df.head(3)

Total null values 3


,user_id,max_number_of_orders,mean_number_of_orders,min_number_of_orders,average_user_product,max_order_day,max_order_by_hour,total_reorders,total_non_reorders,reorder_ratio
0,1,10,5,1,4.222656,4,8,27.0,11.0,0.710449
1,2,14,7,1,10.000000,2,9,52.0,58.0,0.472656
2,3,12,7,2,5.144531,0,16,27.0,9.0,0.750000


In [16]:
pickle.dump((user_df),open('/content/drive/MyDrive/instacart-market-basket-analysis/user_df.pkl','wb'))


In [17]:
user_df = pickle.load(open('/content/drive/MyDrive/instacart-market-basket-analysis/user_df.pkl','rb'))

<h2> Deleting DataFrame which are not more needed</h2>

In [18]:
del average_products, day_of_week, hour_of_day, reorder_by_user, prior
user_df = user_df.fillna(0)
print("Total null values", user_df.isnull().any().sum())

Total null values 0


<h2>Product DataFrame</h2>

1. Number Of Times Product Reordered

In [19]:
product_df = pd.DataFrame()
product_df['times_product_ordered'] = prior_order_df.groupby(by='product_id')['order_id'].aggregate('count').astype(np.uint16)
print("Total null values", product_df.isnull().any().sum().sum())
product_df.head(3)

Total null values 0


,times_product_ordered
product_id,
1,930
2,36
3,126


<h2>Product Reorder Features</h2>

1. Product Reorder Ratio
2. Time Product Not Reordered

In [20]:
reorder_products = pd.DataFrame()
reorder_products['product_reorder_ratio'] = prior_order_df.groupby('product_id')['reordered'].aggregate('mean').astype(np.float16)
reorder_products['product_not_reorder_time'] = prior_order_df[prior_order_df['reordered']==0].groupby('product_id')['reordered'].aggregate('count').astype(np.uint8)
print("Total null values",reorder_products.isnull().any().sum().sum())

reorder_products.head(3)

Total null values 1


,product_reorder_ratio,product_not_reorder_time
product_id,,
1,0.607422,109.0
2,0.111084,32.0
3,0.793457,26.0


<h2>Add To Cart Features</h2>

1. Average Position Of Product On Cart.
2. Late Position Of Product On Cart.
3. Early Position Of Product On Cart.

In [21]:
add_to_cart = pd.DataFrame()
add_to_cart['average_product_cart_position'] = prior_order_df.groupby(by='product_id')['add_to_cart_order'].aggregate('mean').astype(np.float16)
add_to_cart['late_product_cart_position'] = prior_order_df.groupby(by='product_id')['add_to_cart_order'].aggregate('max').astype(np.float16)
add_to_cart['early_product_cart_position'] = prior_order_df.groupby(by='product_id')['add_to_cart_order'].aggregate('min').astype(np.float16)
print("Total null values",add_to_cart.isnull().any().sum().sum())
add_to_cart.head(3)

Total null values 0


,average_product_cart_position,late_product_cart_position,early_product_cart_position
product_id,,,
1,5.640625,36.0,1.0
2,10.445312,27.0,1.0
3,6.261719,41.0,1.0


<h2>Day Of Week Features</h2>

1. Average Nummber Of Product Order Day Wise
2. Earliest Week Day Product Ordered.
3. Late Week Day Product Ordered.

In [22]:
dow_product = pd.DataFrame()
dow_product['average_product_order_day'] = prior_order_df.groupby(by='product_id')['order_dow'].aggregate('mean').astype(np.float16)
dow_product['late_product_order_day'] = prior_order_df.groupby(by='product_id')['order_dow'].aggregate('max').astype(np.float16)
dow_product['early_product_order_day'] = prior_order_df.groupby(by='product_id')['order_dow'].aggregate('min').astype(np.float16)
print("Total null values",dow_product.isnull().any().sum().sum())
dow_product.head(3)

Total null values 0


,average_product_order_day,late_product_order_day,early_product_order_day
product_id,,,
1,2.804688,6.0,0.0
2,2.583984,6.0,0.0
3,2.564453,6.0,0.0


<h2>Order Hour Of Day Features</h2>

1. Average Time Product Ordered.
2. Late Time Of Product Ordered.
3. Earliest Time Of Product Ordered.

In [23]:
order_hour_of_day_product = pd.DataFrame()
order_hour_of_day_product['average_product_hour'] = prior_order_df.groupby(by='product_id')['order_hour_of_day'].aggregate('mean').astype(np.float16)
order_hour_of_day_product['late_product_hour'] = prior_order_df.groupby(by='product_id')['order_hour_of_day'].aggregate('max').astype(np.float16)
order_hour_of_day_product['early_product_hour'] = prior_order_df.groupby(by='product_id')['order_hour_of_day'].aggregate('min').astype(np.float16)
print("Total null values",order_hour_of_day_product.isnull().any().sum().sum())
order_hour_of_day_product.head(3)

Total null values 0


,average_product_hour,late_product_hour,early_product_hour
product_id,,,
1,13.359375,23.0,0.0
2,14.554688,22.0,7.0
3,11.867188,23.0,0.0


<h1>Merging all the product features on 'product_id'</h1>

In [24]:
product_df = pd.merge(product_df, reorder_products, on='product_id', how='left')
product_df = pd.merge(product_df, add_to_cart, on='product_id', how='left')
product_df = pd.merge(product_df, dow_product, on='product_id', how='left')
product_df = pd.merge(product_df, order_hour_of_day_product, on='product_id', how='left')
print("Total null values",product_df.isnull().any().sum().sum())
product_df.head(3)

Total null values 1


,times_product_ordered,product_reorder_ratio,product_not_reorder_time,average_product_cart_position,late_product_cart_position,early_product_cart_position,average_product_order_day,late_product_order_day,early_product_order_day,average_product_hour,late_product_hour,early_product_hour
product_id,,,,,,,,,,,,
1,930,0.607422,109.0,5.640625,36.0,1.0,2.804688,6.0,0.0,13.359375,23.0,0.0
2,36,0.111084,32.0,10.445312,27.0,1.0,2.583984,6.0,0.0,14.554688,22.0,7.0
3,126,0.793457,26.0,6.261719,41.0,1.0,2.564453,6.0,0.0,11.867188,23.0,0.0


In [25]:
pickle.dump((product_df),open('/content/drive/MyDrive/instacart-market-basket-analysis/product_df.pkl','wb'))

In [26]:
product_df = pickle.load(open('/content/drive/MyDrive/instacart-market-basket-analysis/product_df.pkl','rb'))

<h2>User Product Features</h2>

1. Times Product Ordered By User

In [27]:
user_product_df = pd.DataFrame()
user_product_df['times_product_by_user'] = prior_order_df.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').astype(np.uint8)
user_product_df = user_product_df.reset_index()
print("Total null values",user_product_df.isnull().any().sum().sum())
user_product_df.head(3)

Total null values 0


,user_id,product_id,times_product_by_user
0,1,196,7
1,1,10258,5
2,1,10326,1


<h2>Product Bought Features</h2>

1. Times Product Bought By User
2. Minimum Times Product Ordered By User

In [28]:
product_bought = pd.DataFrame()
product_bought['times_bought'] = prior_order_df.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').astype(np.uint8)
product_bought['earliest_order_position'] = prior_order_df.groupby(by=['user_id', 'product_id'])['order_number'].aggregate('min').astype(np.uint8)
product_bought = product_bought.reset_index()
print("Total null values",product_bought.isnull().any().sum().sum())
product_bought.head(3)

Total null values 0


,user_id,product_id,times_bought,earliest_order_position
0,1,196,7,1
1,1,10258,5,3
2,1,10326,1,5


<h2>Order By User Features</h2>

1. Total Order By User.

In [29]:
order_by_user = pd.DataFrame()
order_by_user['total_orders_by_user'] = prior_order_df.groupby('user_id')['order_number'].aggregate('max').astype(np.uint16)
order_by_user = order_by_user.reset_index()
print("Total null values",order_by_user.isnull().any().sum().sum())
order_by_user.head(3)

Total null values 0


,user_id,total_orders_by_user
0,1,10
1,2,14
2,3,12


<h2>Merging Bought Product & Order By User</h2>

In [30]:
merged_user_order = pd.merge(order_by_user, product_bought, how='right', on='user_id')
print("Total null values",merged_user_order.isnull().any().sum().sum())
merged_user_order.head(3)

Total null values 0


,user_id,total_orders_by_user,product_id,times_bought,earliest_order_position
0,1,10,196,7,1
1,1,10,10258,5,3
2,1,10,10326,1,5


<h2>Order Range Feature</h2>


In [31]:
merged_user_order['order_range'] = merged_user_order['total_orders_by_user'] - merged_user_order['earliest_order_position'] + 1
print("Total null values",merged_user_order.isnull().any().sum().sum())
merged_user_order.head(3)

Total null values 0


,user_id,total_orders_by_user,product_id,times_bought,earliest_order_position,order_range
0,1,10,196,7,1,10
1,1,10,10258,5,3,8
2,1,10,10326,1,5,6


<h2>Merging User Product Features On 'user_id' and 'product_id'</h2>

1. Reorder ratio of product by user

In [32]:
user_product_merged_df = pd.merge(merged_user_order, user_product_df, on=['user_id', 'product_id'], how='left')
user_product_merged_df['user_pro_reorder_ratio'] = user_product_merged_df['times_bought'] / user_product_merged_df['order_range']
print("Total null values",user_product_merged_df.isnull().any().sum().sum())
user_product_merged_df.head(3)

Total null values 0


,user_id,total_orders_by_user,product_id,times_bought,earliest_order_position,order_range,times_product_by_user,user_pro_reorder_ratio
0,1,10,196,7,1,10,7,0.700000
1,1,10,10258,5,3,8,5,0.625000
2,1,10,10326,1,5,6,1,0.166667


<h2>Dropping Features Which Aren't Required</h2>

In [33]:
user_product_merged_df = user_product_merged_df.drop(['total_orders_by_user','order_range','times_product_by_user','earliest_order_position','times_bought'],axis=1)
user_product_merged_df.head(3)

,user_id,product_id,user_pro_reorder_ratio
0,1,196,0.700000
1,1,10258,0.625000
2,1,10326,0.166667


<h2>Dropping Features From Product Bought</h2>

In [34]:
product_bought.drop(['earliest_order_position'],axis=1, inplace=True)
product_bought.head(3)

,user_id,product_id,times_bought
0,1,196,7
1,1,10258,5
2,1,10326,1


<h2>Merging Bought Product With User Product Features</h2>

In [35]:
product_bought = pd.merge(product_bought, user_product_merged_df, on=['user_id','product_id'], how='left')
print("Total null values",product_bought.isnull().any().sum().sum())
product_bought.head(3)

Total null values 0


,user_id,product_id,times_bought,user_pro_reorder_ratio
0,1,196,7,0.700000
1,1,10258,5,0.625000
2,1,10326,1,0.166667


<h2>Order Number Back Features</h2>

1. Order Number Back: Times Product Bought By User In Recent Orders

In [36]:
prior_order_df['order_number_back'] = (prior_order_df.groupby(by=['user_id'])['order_number'].transform(max) - prior_order_df['order_number'] + 1).astype(np.uint8)
print("Total null values", prior_order_df.isnull().any().sum().sum())
prior_order_df.head(3)

Total null values 0


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
0,3148837,141850,prior,23,1,14,5.0,47209,3,1,22
1,3148837,141850,prior,23,1,14,5.0,39619,2,1,22
2,3148837,141850,prior,23,1,14,5.0,21709,4,1,22


<h2>Times Product Bought By Use In Last 10 Orders</h2>

In [37]:
temp = prior_order_df.loc[prior_order_df['order_number_back'] <= 10]
temp.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
5,1185907,147186,prior,7,2,13,30.0,5258,8,0,2
6,1185907,147186,prior,7,2,13,30.0,25879,1,0,2
7,1185907,147186,prior,7,2,13,30.0,11941,6,0,2


<h2>Recent Ten Orders</h2>

In [38]:
recent_orders = pd.DataFrame()
recent_orders['recent_ten'] = (temp.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count')).astype(np.uint16)
recent_orders = recent_orders.reset_index()
recent_orders.head(3)

,user_id,product_id,recent_ten
0,1,196,7
1,1,10258,5
2,1,10326,1


<h2>Recent Ten Order Ratio</h2>

In [39]:
recent_orders['recent_ten_ratio'] = (recent_orders['recent_ten'] / 10.0).astype(np.float16)
recent_orders.head(3)

,user_id,product_id,recent_ten,recent_ten_ratio
0,1,196,7,0.700195
1,1,10258,5,0.500000
2,1,10326,1,0.099976


<h2>Merging Product Bought & Recent Orders</h2>

In [40]:
product_bought = pd.merge(product_bought, recent_orders, on=['user_id', 'product_id'], how='left')
del temp, recent_orders
print("Total null",product_bought.isnull().any().sum().sum())
product_bought.head(3)

Total null 2


,user_id,product_id,times_bought,user_pro_reorder_ratio,recent_ten,recent_ten_ratio
0,1,196,7,0.700000,7.0,0.700195
1,1,10258,5,0.625000,5.0,0.500000
2,1,10326,1,0.166667,1.0,0.099976


<h2>Since there are null values so filling them with 0</h2>

In [41]:
product_bought = product_bought.fillna(0)
pickle.dump((product_bought),open('/content/drive/MyDrive/instacart-market-basket-analysis/product_bought.pkl','wb'))
product_bought = pickle.load(open('/content/drive/MyDrive/instacart-market-basket-analysis/product_bought.pkl','rb'))
product_bought.head(3)

,user_id,product_id,times_bought,user_pro_reorder_ratio,recent_ten,recent_ten_ratio
0,1,196,7,0.700000,7.0,0.700195
1,1,10258,5,0.625000,5.0,0.500000
2,1,10326,1,0.166667,1.0,0.099976


<h2>Merging DataFrame Which Contains All Features</h2>

In [42]:
final_data = pd.merge(product_bought, user_df, on='user_id', how='left')
final_data = pd.merge(final_data, product_df, on='product_id', how='left')
print("Total null values",final_data.isnull().any().sum().sum())
final_data.head(3)

Total null values 1


,user_id,product_id,times_bought,user_pro_reorder_ratio,recent_ten,recent_ten_ratio,max_number_of_orders,mean_number_of_orders,min_number_of_orders,average_user_product,...,product_not_reorder_time,average_product_cart_position,late_product_cart_position,early_product_cart_position,average_product_order_day,late_product_order_day,early_product_order_day,average_product_hour,late_product_hour,early_product_hour
0,1,196,7,0.700000,7.0,0.700195,10,5,1,4.222656,...,104.0,3.685547,60.0,1.0,2.898438,6.0,0.0,12.53125,23.0,0.0
1,1,10258,5,0.625000,5.0,0.500000,10,5,1,4.222656,...,6.0,4.203125,27.0,1.0,2.750000,6.0,0.0,12.31250,23.0,0.0
2,1,10326,1,0.166667,1.0,0.099976,10,5,1,4.222656,...,179.0,4.210938,33.0,1.0,2.755859,6.0,0.0,12.81250,23.0,0.0


In [43]:
del product_bought, product_df, user_df

In [44]:
pickle.dump((final_data),open('/content/drive/MyDrive/instacart-market-basket-analysis/final_data.pkl','wb'))
final_data = pickle.load(open('/content/drive/MyDrive/instacart-market-basket-analysis/final_data.pkl','rb'))

<h1>Creating Train Test Data</h1>

In [45]:
train_test_data = orders.loc[((orders['eval_set'] == 'train') | (orders['eval_set'] == 'test')), ['user_id', 'eval_set', 'order_id']]
train_test_data.head()

,user_id,eval_set,order_id
1863274,111823,train,212400
3035931,183161,test,3189541
2979368,179806,train,3313144
96396,5828,train,63759
1553809,93324,test,2212869


In [46]:
final_data = pd.merge(final_data, train_test_data, on='user_id', how='left')
print("Total null values",final_data.isnull().any().sum().sum())
final_data.head(3)

Total null values 3


,user_id,product_id,times_bought,user_pro_reorder_ratio,recent_ten,recent_ten_ratio,max_number_of_orders,mean_number_of_orders,min_number_of_orders,average_user_product,...,late_product_cart_position,early_product_cart_position,average_product_order_day,late_product_order_day,early_product_order_day,average_product_hour,late_product_hour,early_product_hour,eval_set,order_id
0,1,196,7,0.700000,7.0,0.700195,10,5,1,4.222656,...,60.0,1.0,2.898438,6.0,0.0,12.53125,23.0,0.0,train,1187899.0
1,1,10258,5,0.625000,5.0,0.500000,10,5,1,4.222656,...,27.0,1.0,2.750000,6.0,0.0,12.31250,23.0,0.0,train,1187899.0
2,1,10326,1,0.166667,1.0,0.099976,10,5,1,4.222656,...,33.0,1.0,2.755859,6.0,0.0,12.81250,23.0,0.0,train,1187899.0


In [47]:
train_data = final_data.loc[final_data['eval_set']=='train']
test_data = final_data.loc[final_data['eval_set']=='test']

In [48]:
train_data = pd.merge(train_data, train[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left')
train_data.head(3)

,user_id,product_id,times_bought,user_pro_reorder_ratio,recent_ten,recent_ten_ratio,max_number_of_orders,mean_number_of_orders,min_number_of_orders,average_user_product,...,early_product_cart_position,average_product_order_day,late_product_order_day,early_product_order_day,average_product_hour,late_product_hour,early_product_hour,eval_set,order_id,reordered
0,1,196,7,0.700000,7.0,0.700195,10,5,1,4.222656,...,1.0,2.898438,6.0,0.0,12.53125,23.0,0.0,train,1187899.0,1.0
1,1,10258,5,0.625000,5.0,0.500000,10,5,1,4.222656,...,1.0,2.750000,6.0,0.0,12.31250,23.0,0.0,train,1187899.0,1.0
2,1,10326,1,0.166667,1.0,0.099976,10,5,1,4.222656,...,1.0,2.755859,6.0,0.0,12.81250,23.0,0.0,train,1187899.0,NaN


In [49]:
train_data['reordered'].fillna(0, inplace=True)
print("Null values", train_data.isnull().any().sum().sum())

Null values 1


In [50]:
train_data = train_data.set_index(['user_id', 'product_id'])
test_data = test_data.set_index(['user_id', 'product_id'])

train_data.drop(['eval_set', 'order_id'], axis=1, inplace=True)
test_data.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [51]:
print("Train Data Shape", train_data.shape)
print("Test Data Shape", test_data.shape)

Train Data Shape (3780905, 26)
Test Data Shape (2158208, 25)


In [52]:
pickle.dump((train_data),open('/content/drive/MyDrive/instacart-market-basket-analysis/final_data_trainn.pkl','wb'))
pickle.dump((test_data),open('/content/drive/MyDrive/instacart-market-basket-analysis/final_data_test.pkl','wb'))